In [1]:
import os
import pandas as pd
%matplotlib inline

In [2]:
direct ='data/acs_1_yr_vet_status_est/'
ext = '.csv'

acs_dict = {
}

csvs = [i for i in os.listdir(direct) if os.path.splitext(i)[1] == ext]
# dd = 'data/acs_1_yr_vet_status_est/vets_20'
#Store data directory path as variable dd
# acs_dict = {
#     "data/acs_1_yr_vet_status_est/" : "",
#     "" : "",
# }

In [3]:
for f in csvs:
    with open(os.path.join(direct,f)) as file_object:
        acs_dict[f] = file_object.read()

In [4]:
for i in acs_dict:
    print(i, acs_dict[i])

vets_2005_acs.csv GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC01_EST_VC03,HC01_MOE_VC03,HC02_EST_VC03,HC02_MOE_VC03,HC03_EST_VC03,HC03_MOE_VC03,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,HC02_MOE_VC04,HC03_EST_VC04,HC03_MOE_VC04,HC01_EST_VC05,HC01_MOE_VC05,HC02_EST_VC05,HC02_MOE_VC05,HC03_EST_VC05,HC03_MOE_VC05,HC01_EST_VC06,HC01_MOE_VC06,HC02_EST_VC06,HC02_MOE_VC06,HC03_EST_VC06,HC03_MOE_VC06,HC01_EST_VC08,HC01_MOE_VC08,HC02_EST_VC08,HC02_MOE_VC08,HC03_EST_VC08,HC03_MOE_VC08,HC01_EST_VC09,HC01_MOE_VC09,HC02_EST_VC09,HC02_MOE_VC09,HC03_EST_VC09,HC03_MOE_VC09,HC01_EST_VC11,HC01_MOE_VC11,HC02_EST_VC11,HC02_MOE_VC11,HC03_EST_VC11,HC03_MOE_VC11,HC01_EST_VC12,HC01_MOE_VC12,HC02_EST_VC12,HC02_MOE_VC12,HC03_EST_VC12,HC03_MOE_VC12,HC01_EST_VC13,HC01_MOE_VC13,HC02_EST_VC13,HC02_MOE_VC13,HC03_EST_VC13,HC03_MOE_VC13,HC01_EST_VC14,HC01_MOE_VC14,HC02_EST_VC14,HC02_MOE_VC14,HC03_EST_VC14,HC03_MOE_VC14,HC01_EST_VC15,HC01_MOE_VC